# Assignment 3: Hidden Markov Models for POS Tagging and Text Generation
## CNG463 - Introduction to Natural Language Processing
### METU NCC Computer Engineering | Fall 2025-26

**Student Name:** Süleyman Çoban
**Student ID:**  2584886
**Due Date:** 14 December 2025 (Sunday) before midnight

---

## Overview

This assignment focuses on:
1. Building **supervised**, **unsupervised**, and **semi-supervised** HMM models for Part-of-Speech (POS) tagging
2. Implementing **5-fold cross-validation** to evaluate model performance
3. Comparing the three training approaches using per-tag and overall accuracy
4. Using HMMs for **creative text generation** with temperature-based sampling

**Note:** You will use the Brown corpus with universal POS tagset. Start with 5000 sentences for debugging, then run on the full corpus (or as much as Colab can handle).

**Grading:**
- Helper Functions: **10 pts**
  - `remove_tags()`: 2 pts
  - `evaluate_tagger()`: 5 pts
  - Results display: 3 pts
- Semi-supervised HMM: **20 pts**
- 5-fold cross-validation: **25 pts**
  - Supervised HMM: 10 pts
  - Unsupervised HMM (Baum-Welch): 10 pts
  - Semi-supervised HMM
  - Evaluate and Accumulate Results: 5 pts
- Report on Results: **5 pts**
- Text Generation: **24 pts**
  - `sample_state()`: 7 pts
  - `sample_word()`: 7 pts
  - `generate_text_from_word()`: 10 pts
- Written Questions (4 × 4 pts): **16 pts**
- **Total: 100 pts**

---

## Pre-Submission Checklist

- [ ] Name and student ID at top
- [ ] No cells are added or removed
- [ ] All TODO sections completed
- [ ] All questions answered
- [ ] Code runs without errors
- [ ] Results tables included
- [ ] Run All before saving

## Setup and Imports

In [ ]:
# Standard libraries
import numpy as np
import random
from collections import defaultdict

# NLTK for corpus and HMM
import nltk
from nltk.tag import hmm

# Scikit-learn for cross-validation
from sklearn.model_selection import KFold

# Set random seed for reproducibility
seed = 42
np.random.seed(seed)
random.seed(seed)

---

# Task 1: HMM-based POS Tagging (72 points)

In this part, you will implement three different approaches to training HMM models for POS tagging:
1. **Supervised learning**: Uses fully labelled data
2. **Unsupervised learning**: Uses unlabelled data with Baum-Welch algorithm
3. **Semi-supervised learning**: Combines a small amount of labelled data with larger unlabelled data

## 1.1: Load the Brown Corpus

Load the Brown corpus with universal POS tagset. Start with 5000 sentences for testing, then increase to the maximum your environment can handle.

In [ ]:
# Download required NLTK data
from nltk.corpus import brown
nltk.download('brown')
nltk.download('universal_tagset')

NUM_SENTENCES = 1000  # Start with 5000 sentences and increase later

all_data = list(brown.tagged_sents(tagset="universal"))[:NUM_SENTENCES]

print(f"Total sentences loaded: {len(all_data)}")
print(f"\nExample sentence:")
print(all_data[0])

## 1.2: Remove Tags (2 points)

Implement the `remove_tags()` function that converts tagged sentences to untagged format required by NLTK's unsupervised training.

**Input:** List of tagged sentences `[[(word, tag), ...], ...]`  
**Output:** List of untagged sentences `[[(word, None), ...], ...]`

In [ ]:
def remove_tags(tagged_sents):
  untagged_data = []

  # 2. Loop through every sentence in the input data
  for sentence in tagged_sents:

    # Create a temporary list for the current sentence
    new_sentence = []

    # 3. Loop through every word pair in the current sentence
    for word, tag in sentence: # tuple unpacking
      # Keep the word, but change the tag to None #this will unpack each tuples in a sentence and replace the tags with none
      new_pair = (word, None)
      new_sentence.append(new_pair)

      # Add the processed sentence to our main list
      untagged_data.append(new_sentence)
  return untagged_data

## 1.3: Evaluate Tagger (5 points)

Implement the `evaluate_tagger()` function that evaluates a trained tagger on test data and returns per-tag accuracy statistics.

**Input:**
- `tagger`: Trained HMM tagger
- `test_data`: List of tagged test sentences

**Output:** Dictionary with per-tag statistics `{tag: {'correct': count, 'total': count}}`

In [ ]:
def evaluate_tagger(tagger, test_data):

    # 1. Initialize tag_stats as defaultdict(lambda: {'correct': 0, 'total': 0})
    tag_stats = defaultdict(lambda: {'correct': 0, 'total': 0})

    # 2. For each sentence in test_data:
    for sentence in test_data:
        #    a. Extract words and true tags
        words = [word for word, tag in sentence]
        true_tags = [tag for word, tag in sentence]

        try:
            #    b. Use tagger.tag(words) to get predicted tags
            predicted_pairs = tagger.tag(words)
            predicted_tags = [tag for word, tag in predicted_pairs]

            #    c. Compare true vs predicted tags
            for true_tag, pred_tag in zip(true_tags, predicted_tags):

                #    d. Update tag_stats accordingly
                tag_stats[true_tag]['total'] += 1

                # increment 'correct' if the prediction is true
                if true_tag == pred_tag:
                    tag_stats[true_tag]['correct'] += 1

        # 3. Handle exceptions (if tagging fails, count all as incorrect)
        except Exception as e:
            for tag in true_tags:
                tag_stats[tag]['total'] += 1

    # 4. Return tag_stats
    return dict(tag_stats)

## 1.4: Semi-supervised HMM Training (20 points)

Implement semi-supervised HMM training that combines a small amount of labelled data (1%) with larger unlabelled data (99%).

**Steps:**
1. Split data: 1% tagged, 99% untagged
2. Train supervised model on 1% tagged data
3. Use this model to initialise Baum-Welch on 99% untagged data
4. Refine with unsupervised learning

In [ ]:
def train_semi_supervised(tagged_data, percent_tagged=1.0):

# 1. Calculate split index
    total_len = len(tagged_data)
    split_idx = int(total_len * (percent_tagged / 100.0))

    #    Ensure at least 1 sentence is tagged
    if split_idx < 1 and total_len > 0:
        split_idx = 1
    print(f"Splitting: {split_idx} tagged, {total_len - split_idx} untagged.")

    # 2. Split data
    tagged_portion = tagged_data[:split_idx]
    untagged_portion = remove_tags(tagged_data[split_idx:])

    # 3. Extract states and symbols from ALL data
    all_states = set()
    all_symbols = set()

    for sent in tagged_data:
        for word, tag in sent:
            all_states.add(tag)
            all_symbols.add(word)

    # 4. Initialize trainer with states and symbols using
    trainer = hmm.HiddenMarkovModelTrainer(
        states=list(all_states),
        symbols=list(all_symbols)
    )

    # 5. Train supervised model on small tagged data using trainer.train_supervised
    print("Training initial supervised model")
    supervised_model = trainer.train_supervised(tagged_portion)

    # 6. Refine with Baum-Welch on untagged data using trainer.train_unsupervised
    if len(untagged_portion) > 0:
        print("Step 2: Refining with unsupervised learning")
        refined_model = trainer.train_unsupervised(
            untagged_portion,
            model=supervised_model,
            max_iterations=5
        )
        # 7. Return refined_tagger (or error if no untagged data or no tagged data)
        return refined_model
    else:
        raise ValueError("No untagged Data")

## 1.5: 5-Fold Cross-Validation (25 Points)

Implement 5-fold cross-validation to train and evaluate all three models. This ensures robust performance estimates.

**Steps:**
1. Split data into 5 folds
2. For each fold:
   - Train
    - supervised (`trainer.train_supervised()`)
    - unsupervised (`trainer.train_unsupervised()`)
    - semi-supervised models (`train_semi_supervised()`)
   - Evaluate on test fold
   - Accumulate results
3. Calculate average accuracy across all folds

In [ ]:
# Prepare for 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=seed)

# Storage for results across folds
results = {
    'unsupervised': defaultdict(lambda: {'correct': 0, 'total': 0}),
    'supervised': defaultdict(lambda: {'correct': 0, 'total': 0}),
    'semi_supervised': defaultdict(lambda: {'correct': 0, 'total': 0})
}

fold_num = 0

# TODO: Complete the cross-validation loop

for train_idx, test_idx in kf.split(all_data):
    print(f"{'='*60}")
    print(f"FOLD {fold_num}")
    print(f"{'='*60}")
    fold_num += 1

    # Split data to train_data and test_data
    train_data = [all_data[i] for i in train_idx]
    test_data = [all_data[i] for i in test_idx]


    print(f"Training set: {len(train_data)} sentences")
    print(f"Test set: {len(test_data)} sentences")

    # Extract all_tags and all_symbols from training data
    current_states = set()
    current_symbols = set()
    for sent in train_data:
        for word, tag in sent:
            current_states.add(tag)
            current_symbols.add(word)

    # Convert to lists for the trainer
    list_states = list(current_states)
    list_symbols = list(current_symbols)

    # 1. Unsupervised HMM (Baum-Welch)
    print("\n1. Training Unsupervised HMM (Baum-Welch)...")
    try:
        # 1.1 Convert to untagged format
        untagged_train = remove_tags(train_data)

        # 1.2 Initialize trainer with states and symbols
        trainer_unsup = hmm.HiddenMarkovModelTrainer(states=list_states, symbols=list_symbols)

        # 1.3 Train unsupervised
        tagger_unsup = trainer_unsup.train_unsupervised(untagged_train, max_iterations=5)

        # 1.4 Evaluate unsupervised tagger and update the results dict
        fold_stats = evaluate_tagger(tagger_unsup, test_data)

        # Accumulate results
        for tag, counts in fold_stats.items():
            results['unsupervised'][tag]['correct'] += counts['correct']
            results['unsupervised'][tag]['total'] += counts['total']

        print("Unsupervised training complete")
    except Exception as e:
        print(f"ERROR: Unsupervised training failed: {e}")
        import traceback
        traceback.print_exc()

    # 2. Supervised HMM
    print("\n2. Training Supervised HMM...")
    try:
        # 1.1 Initialize trainer with states and symbols
        trainer_sup = hmm.HiddenMarkovModelTrainer(states=list_states, symbols=list_symbols)

        # 1.2 Train supervised
        tagger_sup = trainer_sup.train_supervised(train_data)

        # 1.3 Evaluate supervised tagger and update the results dict
        fold_stats = evaluate_tagger(tagger_sup, test_data)

        # Accumulate results
        for tag, counts in fold_stats.items():
            results['supervised'][tag]['correct'] += counts['correct']
            results['supervised'][tag]['total'] += counts['total']

        print("Supervised training complete")
    except Exception as e:
        print(f"ERROR: Supervised training failed: {e}")
        import traceback
        traceback.print_exc()

    # 3. Semi-supervised HMM (1% tagged, 99% untagged)
    print("\n3. Training Semi-Supervised HMM (1% tagged, 99% untagged)...")
    try:
        # 1.1 Train semi-supervised
        tagger_semi_sup = train_semi_supervised(train_data, percent_tagged=1.0)

        # 1.3 Evaluate semi-supervised tagger and update the results dict
        fold_stats = evaluate_tagger(tagger_semi_sup, test_data)

        # Accumulate results
        for tag, counts in fold_stats.items():
            results['semi_supervised'][tag]['correct'] += counts['correct']
            results['semi_supervised'][tag]['total'] += counts['total']

        print("Semi-supervised training complete")
    except Exception as e:
        print(f"ERROR: Semi-supervised training failed: {e}")
        import traceback
        traceback.print_exc()


FOLD 0
Training set: 800 sentences
Test set: 200 sentences

1. Training Unsupervised HMM (Baum-Welch)...
iteration 0 logprob -6122213.149708354
iteration 1 logprob -4747973.557405709
iteration 2 logprob -4739232.009649948
iteration 3 logprob -4730584.100365082
iteration 4 logprob -4720077.520962452
Unsupervised training complete

2. Training Supervised HMM...


/usr/local/lib/python3.12/dist-packages/nltk/tag/hmm.py:333: RuntimeWarning: overflow encountered in cast
  X[i, j] = self._transitions[si].logprob(self._states[j])
/usr/local/lib/python3.12/dist-packages/nltk/tag/hmm.py:335: RuntimeWarning: overflow encountered in cast
  O[i, k] = self._output_logprob(si, self._symbols[k])
/usr/local/lib/python3.12/dist-packages/nltk/tag/hmm.py:331: RuntimeWarning: overflow encountered in cast
  P[i] = self._priors.logprob(si)
/usr/local/lib/python3.12/dist-packages/nltk/tag/hmm.py:363: RuntimeWarning: overflow encountered in cast
  O[i, k] = self._output_logprob(si, self._symbols[k])


Supervised training complete

3. Training Semi-Supervised HMM (1% tagged, 99% untagged)...
Splitting: 8 tagged, 792 untagged.
Training initial supervised model
Step 2: Refining with unsupervised learning


**Question 1.1:** In an unsupervised HMM with 12 hidden states, you replace the intended PoS tags with meaningless labels like S1–S12. How would this change affect the model’s learned behaviour and its evaluation accuracy? (4 points, 3-4 sentences)

Model's learning behaviour wouldn't change because it does not learn by labels, it learns by discovering patterns based on the sequence statistics and relations within the data. However, evaluation accuracy would become impossible to calculate directly because if we do not correctly map the meaningless labels we wouldn't know which one was noun or which one was verb. But if the mapping is fine then the evaluation accuracy would be same as the unsupervised HMM trained with meaningful labels.

**Question 1.2:** You initialize a semi-supervised HMM with a fully supervised model and then run Baum–Welch only on unlabeled data. How can this additional unsupervised training change the model’s behaviour and accuracy? (4 points, 3-4 sentences)

The additional unsupervised training allows the model to refine its transition and emission probabilities on a much larger corpus (labeled: %1 and unlabelled: %99). And this helped the model to learn much more usage patterns and handle unseen words better. However, since Baum-Welch prioritizes data likelihood rather than label correctness the models learned states may change from the initial supervised learning which may reduce the true Pos tags.

**Question 1.3:** Consider the following four changes to an unsupervised or semi-supervised HMM for PoS tagging. For each one, state whether it would make training faster, slower, or have no significant effect, and justify your choice in one sentence. (4 points, 4 sentences)
- Reducing the number of hidden states from 12 to 6.
- Initializing the model with a fully supervised HMM instead of random parameters.
- Increasing the size of the unlabeled corpus by a factor of 5.
- Replacing full EM with a fixed small number of EM iterations (e.g., exactly 3 passes).

1) Training would be faster because fewer states mean fewer probability calculations.

2) Training would be faster because starting with better initial model allows Baum-Welch to converge faster (in fewer iterations), as the model will start closer to optimal parameters rather than exploring random starting points.

3) Training would be slower because processing 5 times more sentences per iteration directly increases computational cost.

4) Training would be faster because limiting iterations to 3 forces the model to stop before convergence, which eliminates the additional iterations.  

## 1.6: Display Results (5 points)

Display the average across folds results in a formatted table showing
- tag frequency (%)
- per-tag accuracy (%)
- overall accuracy (%)


1.   List item
2.   List item


for all three models.

In [ ]:
# iterate through each model
model_order = ['supervised', 'unsupervised', 'semi_supervised']

print(f"{'='*80}")
print(f"{'FINAL EVALUATION RESULTS (Average across 5 Folds)':^80}")
print(f"{'='*80}\n")

for model_name in model_order:
    # Check if we have data for a model
    if model_name not in results:
        continue

    data = results[model_name]

    # Calculate statistics
    total_correct = sum(tag_data['correct'] for tag_data in data.values())
    total_tokens = sum(tag_data['total'] for tag_data in data.values())

    # Avoid division by zero
    overall_accuracy = (total_correct / total_tokens * 100) if total_tokens > 0 else 0.0

    print(f"MODEL: {model_name}")
    print(f"{'-'*45}")
    print(f"{'TAG':<12} | {'FREQ (%)':<12} | {'ACCURACY (%)':<12}")
    print(f"{'-'*45}")

    # Calculate per-tag statistics
    sorted_tags = sorted(data.keys())

    for tag in sorted_tags:
        stats = data[tag]
        tag_total = stats['total']
        tag_correct = stats['correct']

        if tag_total > 0:
            # How often does the tag appears
            frequency = (tag_total / total_tokens) * 100

            # what is our correct tag rate
            accuracy = (tag_correct / tag_total) * 100

            print(f"{tag:<12} | {frequency:>10.2f}% | {accuracy:>10.2f}%")
        else:
            print(f"{tag:<12} | {'0.00%':>10} | {'N/A':>10}")

    print(f"{'-'*45}")
    print(f"{'OVERALL':<12} | {'100.00%':>10} | {overall_accuracy:>10.2f}%")
    print("\n" + "="*80 + "\n")

## 1.7: Sample Predictions

Test the models on sample sentences to see how it performs.

In [ ]:
# Print sample predictions from the last fold
print("Sample predictions from last fold:")
sample_sentences = [
    "Today is a good day .",
    "Joe met Joanne in Delhi .",
    "Time flies like an arrow .",
    "The good , the bad , the ugly went to a bar ."
]

print(f"{'='*10} Spervised HMM Tagger {'='*10}")
for sent in sample_sentences:
    try:
        tagged = tagger_sup.tag(sent.split())
        print(f"  {sent}")
        print(f"  → {tagged}\n")
    except Exception as e:
        print(f"  {sent}")
        print(f"ERROR: Tagging failed: {e}\n")

print(f"{'='*10} Unspervised HMM Tagger {'='*10}")
for sent in sample_sentences:
    try:
        tagged = tagger_unsup.tag(sent.split())
        print(f"  {sent}")
        print(f"  → {tagged}\n")
    except Exception as e:
        print(f"  {sent}")
        print(f"ERROR: Tagging failed: {e}\n")

print(f"{'='*10} Semi-spervised HMM Tagger {'='*10}")
for sent in sample_sentences:
    try:
        tagged = tagger_semi_sup.tag(sent.split())
        print(f"  {sent}")
        print(f"  → {tagged}\n")
    except Exception as e:
        print(f"  {sent}")
        print(f"ERROR: Tagging failed: {e}\n")

---

# Task 2: Text Generation with HMMs (24 points)

In this part, you will use a trained HMM to generate text. The idea is to sample from the learned transition and emission probabilities to create new sequences.

## 2.1: Train HMM Model for Generation

First, train a supervised HMM model on the Brown corpus for text generation. We'll normalise words to lowercase for better generation.

In [ ]:
def train_hmm_model(num_sentences=5000):
    """Train an HMM model on Brown corpus for text generation"""
    print("Loading Brown corpus...")
    tagged_sents = list(brown.tagged_sents(tagset="universal"))[:num_sentences]

    print(f"Training HMM on {len(tagged_sents)} sentences...")

    # Extract states and symbols
    all_tags = set()
    all_symbols = set()
    for sent in tagged_sents:
        for word, tag in sent:
            all_tags.add(tag)
            all_symbols.add(word.lower())  # Normalise to lowercase

    # Normalise training data to lowercase
    normalized_sents = []
    for sent in tagged_sents:
        normalized_sents.append([(word.lower(), tag) for word, tag in sent])

    # Train the model
    trainer = hmm.HiddenMarkovModelTrainer(
        states=list(all_tags),
        symbols=list(all_symbols)
    )
    tagger = trainer.train_supervised(normalized_sents)
    print("Training complete!")
    return tagger

# Train the model
tagger_gen = train_hmm_model(num_sentences=20000)

# Show model statistics
print(f"\nModel Statistics:")
print(f"  - Number of states (POS tags): {len(tagger_gen._states)}")
print(f"  - Number of symbols (words): {len(tagger_gen._symbols)}")
print(f"  - States: {', '.join(sorted(tagger_gen._states))}")

## 2.2: Implement State Sampling (7 points)

Implement the `sample_state()` function that samples the next POS tag based on transition probabilities.

**Temperature parameter:** Controls randomness
- Lower temperature (e.g., 0.5): More conservative, follows high-probability transitions
- Higher temperature (e.g., 2.0): More creative, explores diverse transitions

In [ ]:
def sample_state(tagger, current_state, temperature=1.0):

    # 1. Get all possible states from tagger._states
    states = list(tagger._states)

    # 2. Get transition probability from current state
    log_probs = np.array([tagger._transitions[current_state].logprob(state) for state in states])

    # 3. Apply temperature scaling
    if temperature <= 0: temperature = 1e-10
    scaled_log_probs = log_probs / temperature

    # 4. Convert from log2 probabilities to regular probabilities
    probs = 2.0 ** scaled_log_probs

    # 5. Normalise probabilities to sum to 1
    total_prob = np.sum(probs)

    if total_prob == 0:
        probs = np.ones(len(states)) / len(states)
    else:
        probs = probs / total_prob

    # 6. Sample using np.random.choice(states, p=probs)
    next_state = np.random.choice(states, p=probs)

    return next_state

## 2.3: Implement Word Sampling (7 points)

Implement the `sample_word()` function that samples a word given a POS tag based on emission probabilities.

In [ ]:
def sample_word(tagger, state, temperature=1.0):

# 1. Get all possible symbols (words) from tagger._symbols
    symbols = list(tagger._symbols)

    # 2. For each symbol, get emission probability from state
    probs = np.array([tagger._outputs[state].logprob(symbol) for symbol in symbols])

    # 3. Apply temperature scaling: probs = probs / temperature
    if temperature <= 0: temperature = 1e-10
    probs = probs / temperature

    # 4. Convert from log2 probabilities to regular probabilities: 2 ** probs
    probs = 2.0 ** probs

    # 5. Handle infinities (words with zero probability) using np.nan_to_num
    probs = np.nan_to_num(probs)

    # 6. Normalise probabilities to sum to 1
    total_prob = np.sum(probs)

    #    (if all are zero, use uniform distribution)
    if total_prob == 0:
        probs = np.ones(len(symbols)) / len(symbols)
    else:
        probs = probs / total_prob

    # 7. Sample using np.random.choice(symbols, p=probs)
    word = np.random.choice(symbols, p=probs)

    return word

## 2.4: Implement Text Generation (10 points)

Implement the `generate_text_from_word()` function that generates text starting from a given word.

In [ ]:
def generate_text_from_word(tagger, start_word, length=20, temperature=1.0):

    # 1. Normalise start_word to lowercase
    current_word = start_word.lower()

    # 2. Check if start_word is in vocabulary (tagger._symbols)
    vocab = set(tagger._symbols)

    # - If not, find similar words or use random word
    if current_word not in vocab:
        print(f"Warning: '{current_word}' not in vocabulary. Using a random word.")
        current_word = random.choice(list(vocab))

    # 3. Initialise generated list with start_word
    generated_list = [current_word]

    # 4. Infer the most likely tag for start_word:
    best_state = None
    max_log_prob = -float('inf')

    #    - For each state, get emission probability
    #    - Choose state with highest probability
    for state in tagger._states:
        prob = tagger._outputs[state].logprob(current_word)
        if prob > max_log_prob:
            max_log_prob = prob
            best_state = state

    if best_state is None:
        best_state = random.choice(list(tagger._states))

    current_state = best_state

    # 5. Generate subsequent words in a loop
    for _ in range(length - 1):

        #    - Sample next state using sample_state()
        next_state = sample_state(tagger, current_state, temperature)

        #    - Sample word from that state using sample_word()
        next_word = sample_word(tagger, next_state, temperature)

        #    - Append word to generated list
        generated_list.append(next_word)
        current_state = next_state

    # 6. Return generated list
    return generated_list

## 2.6: Test Text Generation

Now test your text generation functions with different starting words and temperatures.

In [ ]:
# Test with example words
print("="*70)
print("EXAMPLE GENERATIONS")
print("="*70)

example_words = ["the", "dog", "running", "beautiful", "computer", "yesterday"]

for word in example_words:
    print(f"\n--- Starting with '{word}' ---")
    try:
        words = generate_text_from_word(tagger_gen, word, length=15, temperature=1.0)
        print(" ".join(words))
    except Exception as e:
        print(f"Error: {e}")

In [ ]:
# Test different temperatures
print("\n" + "="*70)
print("TEMPERATURE COMPARISON")
print("="*70)

test_word = "the"
temperatures = [("Conservative", 0.5), ("Balanced", 1.0), ("Creative", 2.0)]

for temp_name, temp_value in temperatures:
    print(f"\n{temp_name} (temp={temp_value}):")
    words = generate_text_from_word(tagger_gen, test_word, length=20, temperature=temp_value)
    print(" ".join(words))

**Question 2.1:** How does the temperature parameter affect the quality and creativity of generated text? Provide two specific examples from your outputs. (4 points, 5-6 sentences)

**[YOUR ANSWER HERE]**

# Convert Your Colab Notebook to PDF

### Step 1: Download Your Notebook
- Go to **File → Download → Download .ipynb**
- Save the file to your computer

### Step 2: Upload to Colab
- Click the **📁 folder icon** on the left sidebar
- Click the **upload button**
- Select your downloaded .ipynb file
- Wait for the upload to complete

### Step 3: Run the Code Below
- **Uncomment the cell below** and run the cell
- This will take about 1-2 minutes to install required packages

### Step 4: Enter Notebook Name
- When prompted, type your notebook name (e.g.`gs_000000_as2.ipynb`)
- Press Enter

### The PDF will automatically download to your computer

In [ ]:
# # Install required packages (this takes about 30 seconds)
# print("Installing PDF converter... please wait...")
# !apt-get update -qq
# !apt-get install -y texlive-xetex texlive-fonts-recommended texlive-plain-generic pandoc > /dev/null 2>&1
# !pip install -q nbconvert

# print("\n" + "="*50)
# print("COLAB NOTEBOOK TO PDF CONVERTER")
# print("="*50)
# print("\nSTEP 1: Download your notebook")
# print("- Go to File → Download → Download .ipynb")
# print("- Save it to your computer")
# print("\nSTEP 2: Upload it here")
# print("- Click the folder icon on the left (📁)")
# print("- Click the upload button and select your .ipynb file")
# print("- Wait for upload to complete")
# print("\nSTEP 3: Enter the filename below")
# print("="*50)

# # Get notebook name from user
# notebook_name = input("\nEnter your notebook name: ")

# # Add .ipynb if missing
# if not notebook_name.endswith('.ipynb'):
#     notebook_name += '.ipynb'

# import os
# notebook_path = f'/content/{notebook_name}'

# # Check if file exists
# if not os.path.exists(notebook_path):
#     print(f"\n⚠ Error: '{notebook_name}' not found in /content/")
#     print("\nMake sure you uploaded the file using the folder icon (📁) on the left!")
# else:
#     print(f"\n✓ Found {notebook_name}")
#     print("Converting to PDF... this may take 1-2 minutes...\n")

#     # Convert the notebook to PDF
#     !jupyter nbconvert --to pdf "{notebook_path}"

#     # Download the PDF
#     from google.colab import files
#     pdf_name = notebook_name.replace('.ipynb', '.pdf')
#     pdf_path = f'/content/{pdf_name}'

#     if os.path.exists(pdf_path):
#         print("✓ SUCCESS! Downloading your PDF now...")
#         files.download(pdf_path)
#         print("\n✓ Done! Check your downloads folder.")
#     else:
#         print("⚠ Error: Could not create PDF")